<a href="https://colab.research.google.com/github/yblee110/jax-flax-book/blob/main/ch03_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jax==0.4.24
!pip install flax==0.7.5
!pip install optax==0.1.7
!pip install datasets
!pip install orbax-checkpoint ==0.4.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: jax
    Found existing installation: jax 0.4.23
    Uninstalling jax-0.4.23:
      Successfully uninstalled jax-0.4.23
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: flax
    Found existing installation: flax 0.8.1
    Uninstalling flax-0.8.1:
      Successfully uninstalled flax-0.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: optax
    Found existing installation: optax 0.2.1
    Uninstalling optax-0.2.1:
      Successfully uninstalled optax-0.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s et

In [2]:
import jax
import flax
import optax
import datasets

import jax.numpy as jnp
from datasets import load_dataset
import flax.linen as nn

print("JAX Version : {}".format(jax.__version__)) #출력 JAX Version : 0.4.20
print("FLAX Version : {}".format(flax.__version__)) #출력 FLAX Version : 0.7.5
print("OPTAX Version : {}".format(optax.__version__))#출력 OPTAX Version : 0.1.7

JAX Version : 0.4.24
FLAX Version : 0.7.5
OPTAX Version : 0.1.7


In [3]:
def get_datasets():
  datasets = load_dataset("mnist")
  datasets = datasets.with_format("jax")
  datasets = {
    "train": {
      "image": datasets["train"]["image"][...,None].astype(jnp.float32)/255,
      "label": datasets["train"]["label"],
    },
    "test": {
      "image": datasets["test"]["image"][...,None].astype(jnp.float32)/255,
      "label": datasets["test"]["label"],
    },
  }
  return datasets['train'], datasets['test']


In [4]:
train_ds, test_ds = get_datasets()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
class CNN(nn.Module):
    num_classes: int


    @nn.compact
    def __call__(self, x, train: bool):
        x = nn.Conv(features=16, kernel_size=(5, 5), strides=(2, 2),
                    padding='VALID')(x)
        x = nn.BatchNorm(use_running_average=not train)(x)
        x = nn.relu(x)
        x = nn.Conv(features=32, kernel_size=(5, 5), strides=(2, 2),
                    padding='VALID')(x)
        x = nn.BatchNorm(use_running_average=not train)(x)
        x = nn.relu(x)
        x = nn.Dropout(rate=0.5, deterministic=not train)(x)
        x = jnp.mean(x, axis=(1, 2))
        x = nn.Dense(features=self.num_classes)(x)
        return x


In [6]:
rng = jax.random.PRNGKey(0)
model = CNN(num_classes=10)
main_key, params_key, dropout_key = jax.random.split(key=rng, num=3)
variables = model.init(params_key, jnp.ones((1, 28, 28, 1)),train=False)
params = variables['params']
batch_stats = variables['batch_stats']

In [7]:
jax.tree_util.tree_map(jnp.shape, variables)

{'batch_stats': {'BatchNorm_0': {'mean': (16,), 'var': (16,)},
  'BatchNorm_1': {'mean': (32,), 'var': (32,)}},
 'params': {'BatchNorm_0': {'bias': (16,), 'scale': (16,)},
  'BatchNorm_1': {'bias': (32,), 'scale': (32,)},
  'Conv_0': {'bias': (16,), 'kernel': (5, 5, 1, 16)},
  'Conv_1': {'bias': (32,), 'kernel': (5, 5, 16, 32)},
  'Dense_0': {'bias': (10,), 'kernel': (32, 10)}}}

In [8]:
def compute_metrics(logits, labels):
  loss = jnp.mean(optax.softmax_cross_entropy(logits, jax.nn.one_hot(labels, num_classes=10)))
  accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
  metrics = {
    'loss': loss,
    'accuracy': accuracy
  }
  return metrics

In [9]:
def create_learning_rate_fn(num_epochs, warmup_epochs, base_learning_rate, steps_per_epoch):
    warmup_fn = optax.linear_schedule(
        init_value=0., end_value=base_learning_rate,
        transition_steps=warmup_epochs * steps_per_epoch)
    cosine_epochs = max(num_epochs - warmup_epochs, 1)

    cosine_fn = optax.cosine_decay_schedule(
        init_value=base_learning_rate,
        decay_steps=cosine_epochs * steps_per_epoch)
    schedule_fn = optax.join_schedules(
        schedules=[warmup_fn, cosine_fn],
        boundaries=[warmup_epochs * steps_per_epoch])

    return schedule_fn

In [10]:
from flax.training import train_state
from typing import Any

train_epoch = 10
warmup_epoch = 2
learning_rate = 0.01
batch_size = 64
eval_batch_size = 100
train_ds_size = len(train_ds['image'])
steps_per_epoch = train_ds_size // batch_size
learning_rate_fn = create_learning_rate_fn(train_epoch, warmup_epoch, learning_rate, steps_per_epoch)

class TrainState(train_state.TrainState):
  batch_stats: Any
  key: jax.random.key

tx = optax.adam(learning_rate=learning_rate_fn)
state = TrainState.create(
  apply_fn=model.apply,
  params=params,
  batch_stats=batch_stats,
  key=dropout_key,
  tx=tx,
)

In [12]:
import orbax.checkpoint
from flax.training import orbax_utils
import os
import shutil

In [16]:
train_path = os.getenv('HOME') + '/cnn/'


ckpt_mgr = orbax.checkpoint.CheckpointManager(
       train_path,
       orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler()),
       orbax.checkpoint.CheckpointManagerOptions(
           create=not (os.path.isdir(train_path)),
           max_to_keep=3,
           step_prefix="model_epoch",
       ),
   )


In [17]:
import functools
@functools.partial(jax.jit, static_argnums=2)
def train_step(state, batch, learning_rate_fn):
  dropout_train_key = jax.random.fold_in(key=dropout_key,
    data=state.step)
  def loss_fn(params):
    logits, updates = state.apply_fn(
    {'params': params, 'batch_stats': state.batch_stats},
    batch['image'], train=True, mutable=['batch_stats'],
    rngs={'dropout': dropout_train_key})
    loss = jnp.mean(optax.softmax_cross_entropy(
      logits=logits,
      labels=jax.nn.one_hot(batch['label'],num_classes=10)))
    return loss, (logits, updates)
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (_, (logits, updates)), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  state = state.replace(batch_stats=updates['batch_stats'])
  metrics = compute_metrics(logits, batch['label'])
  lr = learning_rate_fn(state.step)
  metrics['learning_rate'] = lr
  return state, metrics


In [18]:
@jax.jit
def eval_step(state, batch):
  logits = state.apply_fn(
     {'params': state.params, 'batch_stats': state.batch_stats},
     batch['image'], train=False)
  return compute_metrics(logits, batch['label'])

In [19]:
def train_loop(state, train_ds, batch_size, epoch, learning_rate_fn, rng):
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size
  perms = jax.random.permutation(rng, train_ds_size)
  perms = perms[:steps_per_epoch * batch_size]
  perms = perms.reshape((steps_per_epoch, batch_size))
  batch_metrics = []
  for perm in perms:
    batch = {k: v[perm, ...] for k, v in train_ds.items()}
    state, metrics = train_step(state, batch, learning_rate_fn)
    batch_metrics.append(metrics)
  training_batch_metrics = jax.device_get(batch_metrics)
  training_epoch_metrics = {k: sum([metrics[k] for metrics in
    training_batch_metrics])/steps_per_epoch
    for k in training_batch_metrics[0]}
  print('EPOCH: %d\nTraining loss: %.4f, accuracy: %.2f' % (epoch,
    training_epoch_metrics['loss'],
    training_epoch_metrics['accuracy'] * 100))
  return state


In [20]:
def eval_loop(state, test_ds, batch_size):
  eval_ds_size = test_ds['image'].shape[0]
  steps_per_epoch = eval_ds_size // batch_size
  batch_metrics = []
  for i in range(steps_per_epoch):
    batch = {k: v[i*batch_size:(i+1)*batch_size, ...] for k, v in test_ds.items()}
    metrics = eval_step(state, batch)
    batch_metrics.append(metrics)
  eval_batch_metrics = jax.device_get(batch_metrics)
  eval_batch_metrics = {
    k: sum([metrics[k] for metrics in eval_batch_metrics])/steps_per_epoch
    for k in eval_batch_metrics[0]}

  print('    Eval loss: %.4f, accuracy: %.2f' % (eval_batch_metrics['loss'], eval_batch_metrics['accuracy'] * 100))

In [27]:
for epoch in range(train_epoch):
  rng, key = jax.random.split(rng)
  state = train_loop(state, train_ds, batch_size, epoch,
    learning_rate_fn, rng)
  eval_loop(state, test_ds, eval_batch_size)
  save_args = orbax_utils.save_args_from_target(state)
  ckpt_mgr.save(epoch, state, save_kwargs={"save_args": save_args})

EPOCH: 0
Training loss: 0.1901, accuracy: 94.17
    Eval loss: 0.0851, accuracy: 97.63
EPOCH: 1
Training loss: 0.1551, accuracy: 95.19
    Eval loss: 0.1032, accuracy: 96.96
EPOCH: 2
Training loss: 0.1404, accuracy: 95.61
    Eval loss: 0.1409, accuracy: 95.87
EPOCH: 3
Training loss: 0.1223, accuracy: 96.21
    Eval loss: 0.0610, accuracy: 98.20
EPOCH: 4
Training loss: 0.1080, accuracy: 96.61
    Eval loss: 0.0654, accuracy: 98.07
EPOCH: 5
Training loss: 0.0974, accuracy: 96.97
    Eval loss: 0.0510, accuracy: 98.48
EPOCH: 6
Training loss: 0.0888, accuracy: 97.17
    Eval loss: 0.0483, accuracy: 98.55
EPOCH: 7
Training loss: 0.0844, accuracy: 97.42
    Eval loss: 0.0477, accuracy: 98.56
EPOCH: 8
Training loss: 0.0831, accuracy: 97.45
    Eval loss: 0.0478, accuracy: 98.55
EPOCH: 9
Training loss: 0.0826, accuracy: 97.45
    Eval loss: 0.0478, accuracy: 98.56


In [34]:
raw_restored = ckpt_mgr.restore('9')
raw_restored

{'batch_stats': {'BatchNorm_0': {'mean': array([ 0.75833297,  0.16389968, -0.24726163, -0.46883136, -0.34794843,
          -0.63331527, -0.597255  , -0.5682099 , -0.01700159, -1.0415736 ,
           0.3572851 , -1.5261657 ,  0.9242987 , -0.69794554, -0.03883811,
          -0.7299901 ], dtype=float32),
   'var': array([2.7860937 , 0.47158644, 0.8427777 , 3.1784923 , 2.1192265 ,
          3.3846188 , 3.451998  , 3.8955207 , 1.870601  , 3.1124806 ,
          0.3604667 , 3.5662277 , 2.2233772 , 2.0652814 , 0.63851804,
          2.9032001 ], dtype=float32)},
  'BatchNorm_1': {'mean': array([-3.6724398 ,  0.8711775 , -5.108358  , -0.45900086,  5.7273602 ,
          -7.278806  , -0.35888478, -1.1116356 , -2.7598863 ,  5.8122363 ,
          -2.326713  , -3.2434268 , -2.6020596 , -6.7287574 , -3.027847  ,
           1.3144072 ,  0.5176304 , -0.3516001 ,  0.32117352, -6.6746063 ,
           5.26229   , -4.5569243 ,  1.4865421 , -5.759372  , -4.0041623 ,
           4.6565623 , -0.54005826, -5.486

In [37]:
step = ckpt_mgr.latest_step()  # step = 4
ckpt_mgr.restore(step)

{'batch_stats': {'BatchNorm_0': {'mean': array([ 0.75833297,  0.16389968, -0.24726163, -0.46883136, -0.34794843,
          -0.63331527, -0.597255  , -0.5682099 , -0.01700159, -1.0415736 ,
           0.3572851 , -1.5261657 ,  0.9242987 , -0.69794554, -0.03883811,
          -0.7299901 ], dtype=float32),
   'var': array([2.7860937 , 0.47158644, 0.8427777 , 3.1784923 , 2.1192265 ,
          3.3846188 , 3.451998  , 3.8955207 , 1.870601  , 3.1124806 ,
          0.3604667 , 3.5662277 , 2.2233772 , 2.0652814 , 0.63851804,
          2.9032001 ], dtype=float32)},
  'BatchNorm_1': {'mean': array([-3.6724398 ,  0.8711775 , -5.108358  , -0.45900086,  5.7273602 ,
          -7.278806  , -0.35888478, -1.1116356 , -2.7598863 ,  5.8122363 ,
          -2.326713  , -3.2434268 , -2.6020596 , -6.7287574 , -3.027847  ,
           1.3144072 ,  0.5176304 , -0.3516001 ,  0.32117352, -6.6746063 ,
           5.26229   , -4.5569243 ,  1.4865421 , -5.759372  , -4.0041623 ,
           4.6565623 , -0.54005826, -5.486